Шовене

In [1]:
import math
import statistics as st

p_list = [8.02, 8.16, 3.97, 8.64, 0.84, 4.46, 0.81, 7.74, 8.78, 9.26, 20.46, 29.87, 10.38, 25.71]
# p_list = [15, 16000, 20000, 22000, 24000, 15000, 18000]

iteration_counter = 0

while True and len(p_list) > 1:
    iteration_counter += 1
    stop_iter = True
    p_mean = st.mean(p_list)
    p_stdev = st.stdev(p_list)
    print("Iteration:", iteration_counter)
    for p in list(p_list):
        if p_stdev > 0 and math.erfc(math.fabs(p - p_mean) / p_stdev) < 1 / (2 * len(p_list)):
            print("Anomaly found:", p)
            p_list.remove(p)
            stop_iter = False
    if stop_iter:
        break

print("Result:", p_list)

Iteration: 1
Anomaly found: 29.87
Anomaly found: 25.71
Iteration: 2
Anomaly found: 20.46
Iteration: 3
Anomaly found: 0.84
Anomaly found: 0.81
Iteration: 4
Anomaly found: 3.97
Anomaly found: 4.46
Iteration: 5
Anomaly found: 10.38
Iteration: 6
Anomaly found: 7.74
Anomaly found: 9.26
Iteration: 7
Result: [8.02, 8.16, 8.64, 8.78]


In [4]:
# Скрипт тестирует критерий Шовене для наибольшего и наименьшего значения строки значений р.

# Python 3

# Среднее, стандартное отклонение со смещением и дополненная функция ошибок из пакета scipy

from scipy.stats import tmean, tstd 
from scipy.special import erfc 

# каждая строка набор значений р
p_set = ((8.02, 8.16, 3.97, 8.64, 0.84, 4.46, 0.81, 7.74, 8.78, 9.26, 20.46, 29.87, 10.38, 25.71),
         (8.02, 8.16, 3.97, 8.64, 0.84, 4.46, 0.81, 7.74, 8.78, 9.26, 20.46, 10.38),
         (8.02, 8.16, 3.97, 8.64, 0.84, 4.46, 0.81, 7.74, 8.78, 9.26, 10.38))

for p in p_set:
    n = len(p)
    m = tmean(p)
    std = tstd(p)
    rv = 1 / (2 * n) # правая часть неравенства
    for p_i in (max(p), min(p)):
        erfc_x = erfc(abs(p_i - m) / std)
        print("p =", *p)
        print("{:5} {:2} {:9} {:9} {:9} {:9} {}".format("p_i", "n", "p_mean", "std", "erfc", "r_value", "Chauvenet"))

        print("{:>5.2f} {} {:9.6f} {:9.6f} {:9.6f} {:9.6f} {}".format(p_i, n, m, std, erfc_x, rv, erfc_x < rv))
        print()

p = 8.02 8.16 3.97 8.64 0.84 4.46 0.81 7.74 8.78 9.26 20.46 29.87 10.38 25.71
p_i   n  p_mean    std       erfc      r_value   Chauvenet
29.87 14 10.507143  8.767465  0.001788  0.035714 True

p = 8.02 8.16 3.97 8.64 0.84 4.46 0.81 7.74 8.78 9.26 20.46 29.87 10.38 25.71
p_i   n  p_mean    std       erfc      r_value   Chauvenet
 0.81 14 10.507143  8.767465  0.117777  0.035714 False

p = 8.02 8.16 3.97 8.64 0.84 4.46 0.81 7.74 8.78 9.26 20.46 10.38
p_i   n  p_mean    std       erfc      r_value   Chauvenet
20.46 12  7.626667  5.167001  0.000444  0.041667 True

p = 8.02 8.16 3.97 8.64 0.84 4.46 0.81 7.74 8.78 9.26 20.46 10.38
p_i   n  p_mean    std       erfc      r_value   Chauvenet
 0.81 12  7.626667  5.167001  0.062079  0.041667 False

p = 8.02 8.16 3.97 8.64 0.84 4.46 0.81 7.74 8.78 9.26 10.38
p_i   n  p_mean    std       erfc      r_value   Chauvenet
10.38 11  6.460000  3.376540  0.100624  0.045455 False

p = 8.02 8.16 3.97 8.64 0.84 4.46 0.81 7.74 8.78 9.26 10.38
p_i   n  p_mean    

In [3]:
import pandas as pd
import numpy as np
from scipy.special import erfc

my_set = [8.02, 8.16, 3.97, 8.64, 0.84, 4.46, 0.81, 7.74, 8.78, 9.26, 20.46, 29.87, 10.38, 25.71]

df_my_set = pd.DataFrame(data=my_set).sort_values(0).rename(columns={0:'df_values'}).reset_index().loc[:,['df_values']]

delited_values = []
current_df = df_my_set # делаем копию датасета
iter = 1
while len(current_df)>2:
  current_df['standarted_values'] = erfc((current_df.df_values - current_df.df_values.mean())/current_df.df_values.std()) # добавляем колоку со значениями erfc
  criterion = 1/(2*current_df.df_values.count()) # считаем критерий
  detected_values = current_df.query('standarted_values < @criterion').reset_index().loc[:, 'df_values'].to_list() # отбираем выбросы
  current_df = current_df.query('standarted_values >= @criterion').reset_index().loc[:, ['df_values']] # переопределяем датасет, оставив только не выбросы
  if len(detected_values) == 0: # если выбросов больше нет - выходим из цикла
    break
  else:
    print(f'Iter:{iter}, Delited values: {detected_values}')
    delited_values.extend(detected_values)
    iter += 1
print('-'*40)
print(f'Clean data: {current_df.df_values.to_list()}')

Iter:1, Delited values: [25.71, 29.87]
Iter:2, Delited values: [20.46]
----------------------------------------
Clean data: [0.81, 0.84, 3.97, 4.46, 7.74, 8.02, 8.16, 8.64, 8.78, 9.26, 10.38]


In [7]:
import scipy.special as ss
A = np.array([5, 6, 8, 4, 2, 9, 3, 24])
print(A[ss.erfc(abs(A - np.average(A)) / np.std(A)) > 1 / (len(A) * 2)])

[5 6 8 4 2 9 3]


In [4]:
mass = np.array([8.02, 8.16, 3.97, 8.64, 0.84, 4.46, 0.81, 7.74, 8.78, 9.26, 20.46, 29.87, 10.38, 25.71])

from scipy.special import erfc

func = lambda x: erfc(abs(x - x.mean()) / x.std(ddof=1)) < (x.size ** -1 / 2)

mass = mass[~func(mass)]

mass

array([ 8.02,  8.16,  3.97,  8.64,  0.84,  4.46,  0.81,  7.74,  8.78,
        9.26, 20.46, 10.38])